In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from sklearn.linear_model import LinearRegression

import voteestimator
import numpy as np

## Read

In [ ]:
class AnalysisSetCreator:
    
    def __init__(self, votesmodel='Meindertsma'):

        votesmodels = {'Meindertsma': voteestimator.MeindertsmaVotesEstimator(),
                      'Exponential': voteestimator.ExponentialVotesEstimator()
                      }
        self.votesmodel = votesmodels[votesmodel]
    
    def _combine_data(self, filefolder):
        self.notering = pd.read_parquet(os.path.join(filefolder, 'notering.parquet'))
        self.song = pd.read_parquet(os.path.join(filefolder, 'song.parquet'))
        self.songartist = pd.read_parquet(os.path.join(filefolder, 'songartist.parquet'))
        self.artist = (pd.read_parquet(os.path.join(filefolder, 'artist.parquet')) # TODO: This should not happen here
                          .pipe(self._artist_features)
                        )
        
        df = (self.notering.merge(self.song, left_on='SongID', right_index=True)
                           .merge(self.songartist.reset_index())
                           .merge(self.artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
             )
        return df
    
    def _read_stemperiodes(self, path=os.path.join('Data', 'EindeStemperiode.xlsx')):
        einde_stemperiode = (pd.read_excel(path, engine='openpyxl')
                               .dropna(subset=['EindeStemperiode'])
                               .drop(columns=['Bron'])
                               .sort_values('EindeStemperiode')
                            )
        return einde_stemperiode
    

    def _check_passed_away_during_top2000(self, df, top2000_stemperiodes):
        first_stemperiode = top2000_stemperiodes['EindeStemperiode'].min()
        relevant_date_of_death = first_stemperiode + pd.Timedelta('365 days')
        df['IsOverleden'] = df['Overlijdensdatum'].ge(relevant_date_of_death)
        return df
    
    def _find_next_top2000_after_death(self, df, top2000_stemperiodes):
        not_passed_away_during_top_2000 = df[~df['IsOverleden']].copy()
        passed_away_during_top2000 = (df.loc[df['IsOverleden']]
                                      .sort_values('Overlijdensdatum')
                                      .reset_index()
                                     )

        passed_away_during_top2000 = (pd.merge_asof(passed_away_during_top2000, top2000_stemperiodes,
                                                   left_on='Overlijdensdatum', right_on='EindeStemperiode', direction='forward')
                                     .set_index('ArtistID')
                                     )
        df = pd.concat([not_passed_away_during_top_2000, passed_away_during_top2000], sort=False)
        return df


    def _artist_features(self, df):
        einde_stemperiode = self._read_stemperiodes()
        df = (df                                          
                .pipe(self._check_passed_away_during_top2000, einde_stemperiode)
                .pipe(self._find_next_top2000_after_death, einde_stemperiode)
                .assign(AgePassing = lambda df: df['Overlijdensdatum'].sub(df['Geboortedatum']).dt.days / 365.25,
                        PassingTooEarly = lambda df: df['AgePassing'].sub(80).mul(-1).clip(lower=0),
                        IsDutch = lambda df: df['IsDutch'].astype(int),
                        )
             )
        return df
    
    def _rank_features(self, df):
        return df.assign(PctVotes = lambda df: df['Rank'].apply(self.votesmodel.percentage_of_votes))
    
    
    def _normalize_by_years_before_death(self, df, years_to_normalize=2):        
        mi = pd.MultiIndex.from_product([df.query('IsOverleden')['SongID'].unique(),
                                         df.query('IsOverleden')['YearsSinceOverlijden'].unique(),],
                                        names=['SongID', 'YearsSinceOverlijden'])
        votes_before_death = (pd.DataFrame(index=mi)
                              .join(self.songartist)
                              .join(df.set_index(['SongID', 'YearsSinceOverlijden', 'ArtistID'])[['Year', 'PctVotes']])
                              .join(self.artist[['JaarTop2000']])
                              .join(self.song[['YearMade']])
                              .assign(YearTop2000 = lambda df: df['JaarTop2000'].add(df.index.get_level_values('YearsSinceOverlijden')),
                                      PctVotes = lambda df: np.where(df['YearTop2000'].gt(df['YearMade']) & df['YearTop2000'].le(df['Year'].max()),
                                                             df['PctVotes'].fillna(self.votesmodel.lower_than_2000), np.nan)
                                     )
                             ['PctVotes']
                             .unstack('YearsSinceOverlijden')
                             .loc[:, range(-years_to_normalize, 0)]
                             .mean(axis='columns')
                             .rename('PctVotesBeforeDeath')
                             .reset_index()
                             )
        
        df = df.merge(votes_before_death, how='left')
        return df

    
    def _song_features(self, df):
        
        df = (df.assign(NrArtists = lambda df: df.groupby(['SongID', 'Year'])['Rank'].transform('count'),
                        YearsSinceOverlijden = lambda df: df['Year'].sub(df['JaarTop2000']),
                       )
                .pipe(self._normalize_by_years_before_death)
             )
        return df
    
    def _song_features_after_passing(self, df):
        df = (df.assign(NrsBeforeDeath = lambda df: df.groupby('ArtistID')['ArtistID'].transform('count'),
                        PopularityWithinArtist = lambda df: df.groupby('ArtistID')['PctVotesBeforeDeath'].apply(lambda v: v.div(v.mean())),
                        LogSongPopularityWithinArtist = lambda df: np.log10(df['PopularityWithinArtist']),
                        RecencyWithinArtist = lambda df: df.groupby('ArtistID')['YearMade'].apply(lambda v: v.sub(v.min()).div(v.max() - v.min())),
                        YearsBeforeDeath = lambda df: df['YearMade'].sub(df['JaarTop2000']),
                        Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                        MultiplePerformers = lambda df: df['NrArtists'].gt(1).astype(int),
                        JarenGeleden = lambda df: df['JaarTop2000'].sub(df['JaarTop2000'].max()),
                        )
             )
        return df
    
    def create_analysis_set(self, filefolder):
        df = (self._combine_data(filefolder)
                  .pipe(self._rank_features)
                  .pipe(self._song_features)
                  .query('YearsSinceOverlijden == 0')
                  .query(f'PctVotesBeforeDeath > {self.votesmodel.lower_than_2000}')
                  .pipe(self._song_features_after_passing)
             )
        return df
    
    def create_artist_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = (df.groupby('ArtistID')
                        .agg({'PctVotes': 'sum',
                              'PctVotesBeforeDeath': 'sum',
                               'YearMade': 'last'
                            }
                            )
                        .join(self.artist[['Name', 'IsDutch', 'AgePassing', 'JaarTop2000', 'Overlijdensdatum', 'EindeStemperiode']])
                        .assign(DaysToStemperiode = lambda df: df['Overlijdensdatum'].sub(df['EindeStemperiode']).dt.days,
                                YearsSinceLastHit = lambda df: df['JaarTop2000'].sub(df['YearMade']),
                                LogPopularity = lambda df: np.log10(df['PctVotesBeforeDeath']),
                                LogPopularityNorm = lambda df: df['LogPopularity'].sub(df['LogPopularity'].median()),
                                Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                                LogBoost = lambda df: np.log(df['Boost']),
                                )
                    )
        return df_artist
    
    def create_full_feature_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = self.create_artist_set(filefolder)#.pipe(self._artist_features)
        full_set = (df.merge(df_artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
                      .assign(
                              SongRelativeBoost = lambda df: df['BoostSong'].div(df['BoostArtist']),
                              LogRelativeBoost = lambda df: np.log2(df['SongRelativeBoost']),
                              LogBoost = lambda df: np.log(df['BoostSong']),
                             )
           )
        return full_set

a = AnalysisSetCreator()
df_song = a.create_analysis_set('Data')
df_artist = a.create_artist_set('Data')
df = a.create_full_feature_set('Data')

In [ ]:
df_artist['Boost'].describe()

In [ ]:
df['SongID'].nunique()

In [ ]:
a = AnalysisSetCreator()
full_list = a._combine_data('Data')

In [ ]:
full_list['SongID'].nunique()

In [ ]:
full_list['ArtistID'].nunique()

In [ ]:
df_song['Boost'].describe()

In [ ]:
def calculate_new_position(old_position, votesmodel, boost=1.75):
    old_votes = votesmodel.percentage_of_votes(old_position)
    new_votes = old_votes
    new_position = old_position
    while new_votes < boost * old_votes and new_position >= 1:
        new_position -= 1
        new_votes = votesmodel.percentage_of_votes(new_position)
    return max(new_position, 1)

In [ ]:
positioning = (pd.DataFrame({'OldPosition': range(1, 2001)})
               .assign(NewPosition = lambda df: [calculate_new_position(p, a.votesmodel) for p in df['OldPosition']],
                      FactorRanking = lambda df: df['NewPosition'] / df['OldPosition']))

In [ ]:
positioning.plot(x='OldPosition', y='FactorRanking')

## Create analysis set

## Univariate analysis
* Newsworthyness of artist passing
  * Age when passing away
  * Dutch nationality
  * Popularity
  * Year
  * Recency of hits
* Days to stemperiode

* Song
  * Popularity within artist
  * Recency of song within artist
  
### Newsworthyness

#### Artist age

In [ ]:
def plot_with_trend(df, column, logy=False):
    ycolname = 'LogBoost' if logy else 'Boost'
    preds = LinearRegression().fit(df[[column]], df[ycolname]).predict(df[[column]])
    ax = df.plot(x=column, y=ycolname, kind='scatter', label='Passed away artists', c='grey')
    ax.plot(df[column], preds, 'k', label='Trend')
    return ax

In [ ]:
ax = plot_with_trend(df_artist, 'AgePassing', logy=True)

#### Jaar

In [ ]:
ax = plot_with_trend(df_artist, 'JaarTop2000', logy=False)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_ylim(None, 6)
plt.legend()
plt.tight_layout()
ax.set_frame_on(False)
plt.gcf().savefig('YearEffect.jpg')

#### Nationality

In [ ]:
_ = sns.catplot(x='IsDutch', y='LogBoost', data=df_artist)

#### Popularity

In [ ]:
ax = plot_with_trend(df_artist, 'PctVotes', logy=True)

In [ ]:
plot_with_trend(df_artist, 'LogPopularity', logy=True)

In [ ]:
df_artist.nlargest(5, 'Boost')

In [ ]:
dftest = df_artist.assign(Popularity = lambda df: df['LogPopularity'].sub(df['LogPopularity'].median()),
                            Boost = lambda df: df['Boost'])

preds = LinearRegression().fit(dftest[['Popularity']], dftest['Boost']).predict(dftest[['Popularity']])
ax = dftest.plot(x='Popularity', y='Boost', kind='scatter', c='grey', ylim=(None, 6))
ax.plot(dftest[['Popularity']], preds, 'k', label='trend')
plt.legend()
plt.tight_layout()
plt.gcf().savefig('PopularityEffect.jpg')

In [ ]:
preds

#### Recency

In [ ]:
ax = plot_with_trend(df_artist.query('DaysToStemperiode > -365'), 'DaysToStemperiode', logy=True)

In [ ]:
recency_buckets = (pd.cut(df_artist['DaysToStemperiode'].clip(lower=-365), 6, labels=False, retbins=False, right=False)
                  .map({i: v for i, v in enumerate(range(-333, 0, 60))}))

In [ ]:
df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
ax = df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count']).plot(y='mean', yerr='sem')
plt.xlim(-365, 0)
plt.show()

### Recency of last hit

In [ ]:
ax = plot_with_trend(df_artist, 'YearsSinceLastHit', logy=True)

### Song

### Solo song

In [ ]:
df.groupby(['NrArtists'])['LogBoost'].agg(['mean', 'std', 'sem', 'count'])

In [ ]:
df.groupby(['MultiplePerformers'])['LogBoost'].agg(['mean', 'std', 'sem', 'count'])

### Popularity within artist oeuvre

In [ ]:
ax = plot_with_trend(df, 'PopularityWithinArtist', logy=True)

In [ ]:
ax = plot_with_trend(df, 'LogSongPopularityWithinArtist', logy=True)

### Recency within artist

In [ ]:
ax = plot_with_trend(df.query('NrsBeforeDeath > 2'), 'RecencyWithinArtist', logy=True)

In [ ]:
df_artist.nlargest(5, 'Boost')

In [ ]:
def plot_waterfall(data, color=None, buildup=False, **kwargs):
    '''
    Plot a buildup or builddown waterfall chart from data
    This function was adapted from https://pbpython.com/waterfall-chart.html

    Parameters
    ----------
    data: pd.Series to be shown as waterfall
    color: optionally give color as a list for each bar (to highlight some bars)
    buildup: False (default) for builddown, True for buildup

    Returns
    -------
    ax: Axis object
    data: the data, including a "total"-row
    blank: the size of the blank space before each bar
    '''
    #TODO: add connecting lines
    if color is None:
        color = ['lightgray'] * len(data)

    blank = data.cumsum().shift(1).fillna(0)
    total = data.sum()
    data.loc['Total'] = total
    blank.loc['Total'] = 0
    color = color + ['gray']
    
    step = blank.reset_index(drop=True).repeat(3).shift(-1)
    step[1::3] = np.nan
    
    if buildup:
        data = data[::-1]
        blank = blank[::-1]
        color = color[::-1]

    ax = data.plot(kind='barh', stacked=True, left=blank, color=color, **kwargs)
#     ax.plot(step.values, step.index, 'k--')

    return ax, data, blank

data = pd.cut(df_artist['Boost'], [0, 1, 1.5, 2.5, np.inf],
               labels=['No boost',
                       'Up to 50% more votes',
                       '1.5 - 2.5 x\nas many votes',
                       'More than 2.5\nx as many votes']).value_counts(normalize=True).sort_index(ascending=False)
data.index = data.index.astype(str)
ax, _, _ = plot_waterfall(data, color=['purple', 'purple', 'purple', 'lightgrey'])
import matplotlib.ticker as mtick

ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
ax.set_frame_on(False)
plt.tight_layout()
plt.gcf().savefig('BoostDistribution.jpg')

In [ ]:
df_song[df_song['Title'].eq('Laat me')]

## Multivariate

In [ ]:
import pymc3 as pm
import arviz as az

In [ ]:
passed_away_artists = df['ArtistID'].unique()
artist_lookup = dict(zip(passed_away_artists, range(len(passed_away_artists))))

In [ ]:
artist_vals = df['ArtistID'].replace(artist_lookup).values

In [ ]:
artist_model = (df.assign(ArtistIDModel = lambda df: df['ArtistID'].map(artist_lookup),
                         )
                .sort_values('ArtistIDModel')
                .drop_duplicates(['ArtistIDModel'])
               )

In [ ]:
coords = {"obs_id": np.arange(df.shape[0]), 'Artist': passed_away_artists}
with pm.Model(coords=coords) as multilevel_model:
    artist_idx = pm.Data("artist_idx", artist_vals, dims="obs_id")
    jaren_geleden = pm.Data("jaren_geleden", artist_model['JarenGeleden'], dims='Artist')
    logpopularity = pm.Data('logpopularity', artist_model['LogPopularityNorm'], dims='Artist')
    days_to_stemperiode = pm.Data('days_to_stemperiode', artist_model['DaysToStemperiode'], dims='Artist')
    passing_too_early = pm.Data('passing_too_early', artist_model['PassingTooEarly'], dims='Artist')
    
    multiple_performers = pm.Data('multiple_performers', df['MultiplePerformers'], dims="obs_id")
    popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', df['LogSongPopularityWithinArtist'], dims="obs_id")
    
    # Hyperpriors:
    a = pm.Normal("a", mu=0, sigma=10.0)
    sigma_a = pm.Exponential("sigma_a", 1.0)
    recency_effect_exponent = pm.Normal('recency_effect_exponent', mu=-1.5,sigma=1)
    max_recency_effect = pm.Exponential('max_recency_effect', 1)
    effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=10)
    history_effect = pm.Normal('history_effect', mu=0, sigma=10)
    age_passing_effect = pm.Normal('age_passing_effect', mu=0, sigma=10)
    
    # Expected value per artist:
    mu_artist = (a
                 + effect_popularity * logpopularity
                 + np.exp(10**recency_effect_exponent * days_to_stemperiode) * max_recency_effect
                 + jaren_geleden * history_effect
                 + passing_too_early * age_passing_effect
                )
    
    a_artist = pm.Normal("a_artist", mu=mu_artist, sigma=sigma_a, dims="Artist")

    sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=10.0)
    within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=10.0)
    theta = (a_artist[artist_idx]
             + multiple_performers * sharing_effect
             + within_oeuvre_effect * popularity_within_oeuvre
            )
    # Model error:
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=df['LogBoost'], dims="obs_id")
pm.model_to_graphviz(multilevel_model)

In [ ]:
np.exp(0.001 * 0)

In [ ]:
coords = {"obs_id": np.arange(df.shape[0]), 'Artist': passed_away_artists}
with pm.Model(coords=coords) as multilevel_noncentered_model:
    artist_idx = pm.Data("artist_idx", artist_vals, dims="obs_id")
    jaren_geleden = pm.Data("jaren_geleden", artist_model['JarenGeleden'], dims='Artist')
    logpopularity = pm.Data('logpopularity', artist_model['LogPopularityNorm'], dims='Artist')
    days_to_stemperiode = pm.Data('days_to_stemperiode', artist_model['DaysToStemperiode'], dims='Artist')
    passing_too_early = pm.Data('passing_too_early', artist_model['PassingTooEarly'], dims='Artist')
    is_dutch = pm.Data('is_dutch', artist_model['IsDutchArtist'], dims='Artist')
    
    multiple_performers = pm.Data('multiple_performers', df['MultiplePerformers'], dims="obs_id")
    popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', df['LogSongPopularityWithinArtist'], dims="obs_id")
    
    # Hyperpriors:
    a = pm.Normal("a", mu=0, sigma=2.0)
    sigma_a = pm.Exponential("sigma_a", 1.0)
    
    recency_effect_exponent = pm.Normal('recency_effect_exponent', mu=-1.5,sigma=1)
    max_recency_effect = pm.Exponential('max_recency_effect', 1)
    effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=2)
    history_effect = pm.Normal('history_effect', mu=0, sigma=1)
    age_passing_effect = pm.Normal('age_passing_effect', mu=0, sigma=1)
    is_dutch_effect = pm.Normal('is_dutch_effect', mu=0, sigma=2)
    
    # Expected value per artist:
    mu_artist = (a
                 + effect_popularity * logpopularity
                 + (np.exp(10**recency_effect_exponent * days_to_stemperiode) - np.exp(10**recency_effect_exponent * -365))* max_recency_effect
                 + jaren_geleden * history_effect
                 + passing_too_early * age_passing_effect
                 + is_dutch * is_dutch_effect
                )
    mu_artist = pm.Deterministic("mu_artist", mu_artist, dims='Artist')
    za_artist = pm.Normal("za_artist", mu=0.0, sigma=1.0, dims="Artist")
    a_artist = pm.Deterministic("a_artist", mu_artist + za_artist * sigma_a, dims="Artist")

    sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=2.0)
    within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=2.0)
    theta = (a_artist[artist_idx]
             + multiple_performers * sharing_effect
             + within_oeuvre_effect * popularity_within_oeuvre
            )
    # Model error:
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=df['LogBoost'], dims="obs_id")
pm.model_to_graphviz(multilevel_noncentered_model)

In [ ]:
with pm.Model(coords=coords) as unpooled_model:
#     artist_idx = pm.Data("artist_idx", artist_vals, dims="obs_id")
    jaren_geleden = pm.Data("jaren_geleden", df['JarenGeleden'], dims='obs_id')
    logpopularity = pm.Data('logpopularity', df['LogPopularityNorm'], dims='obs_id')
    days_to_stemperiode = pm.Data('days_to_stemperiode', df['DaysToStemperiode'], dims='obs_id')
    age_passing = pm.Data('age_passing', df['AgePassingArtist'], dims='obs_id')
    
    multiple_performers = pm.Data('multiple_performers', df['MultiplePerformers'], dims="obs_id")
    popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', df['LogSongPopularityWithinArtist'], dims="obs_id")
    
    # Hyperpriors:
    a = pm.Normal("a", mu=0, sigma=10.0)
    recency_effect_exponent = pm.Normal('recency_effect_exponent', mu=-1.5,sigma=1)
    max_recency_effect = pm.Exponential('max_recency_effect', 1)
    effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=10)
    history_effect = pm.Normal('history_effect', mu=0, sigma=10)
    age_passing_effect = pm.Normal('age_passing_effect', mu=0, sigma=10)
    
    # Expected value per artist:
    mu_artist = (a
                 + effect_popularity * logpopularity
                 + np.exp(10**recency_effect_exponent * days_to_stemperiode) * max_recency_effect
                 + jaren_geleden * history_effect
                 + age_passing * age_passing_effect
                )
    
    sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=10.0)
    within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=10.0)
    theta = (mu_artist
             + multiple_performers * sharing_effect
             + within_oeuvre_effect * popularity_within_oeuvre
            )
    # Model error:
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=df['LogBoost'], dims="obs_id")
pm.model_to_graphviz(multilevel_model)

In [ ]:
# with multilevel_model:
#     multilevel_model_idata = pm.sample(10000, tune=3000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.9)


In [ ]:
RANDOM_SEED = 42
with multilevel_noncentered_model:
    multilevel_noncentered_model_idata = pm.sample(20000, tune=10000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.95)

In [ ]:
RANDOM_SEED = 42
with unpooled_model:
    unpooled_model_idata = pm.sample(5000, tune=3000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.95)


In [ ]:
# az.summary(unpooled_model_idata, round_to=3)

In [ ]:
az.summary(multilevel_noncentered_model_idata, var_names=['~za_artist', '~a_artist', '~mu_artist'], round_to=3)

In [ ]:
# az.summary(multilevel_model_idata, var_names=['~a_artist'], round_to=3)

In [ ]:
az.rcParams['plot.max_subplots'] = 100
var_names = [
            '~a_artist',
            '~sigma_a',
            '~sigma',
            '~za_artist',
            '~mu_artist'
            ]
_ = pm.pairplot(multilevel_noncentered_model_idata, var_names=var_names, marginals=True, divergences=True, kind=['scatter', 'kde'])

In [ ]:
_ = az.plot_trace(multilevel_noncentered_model_idata, compact=True)

In [ ]:
az.plot_autocorr(multilevel_noncentered_model_idata, var_names=var_names, combined=True, max_lag=20)
plt.show()

In [ ]:
import seaborn as sns
def plot_correlations(df):
    fig, ax = plt.subplots(figsize=(12, 12))
    ax = sns.heatmap(df.corr(), cmap='RdBu_r', vmin=-0.8, vmax=0.8, annot=True, fmt='.1%', ax=ax, cbar=False)
    return ax

In [ ]:
# parameters = multilevel_model_idata.posterior.to_dataframe().droplevel('Artist').loc[lambda x: ~x.index.duplicated()]
# plot_correlations(parameters)

In [ ]:

parameters = multilevel_noncentered_model_idata.posterior.to_dataframe().droplevel('Artist').loc[lambda x: ~x.index.duplicated()]
# plot_correlations(parameters)
plot_correlations(parameters)

In [ ]:
# parameters = unpooled_model_idata.posterior.to_dataframe()
# plot_correlations(parameters)

In [ ]:
fig, ax = plt.subplots()
multilevel_noncentered_model_idata.posterior = multilevel_noncentered_model_idata.posterior.assign_coords({"N_artist": ("Artist", artist_model['NrsBeforeDeath'])})
# plot means
multilevel_noncentered_model_idata.posterior.mean(dim=("chain", "draw")).plot.scatter(
    x="N_artist", y="a_artist", ax=ax, alpha=0.9
)
ax.hlines(
    multilevel_noncentered_model_idata.posterior.a.median(),
    0.9,
    artist_model['NrsBeforeDeath'].max() + 1,
    alpha=0.4,
    ls="--",
    label="Est. population mean",
)

# plot hdi
hdi = az.hdi(multilevel_noncentered_model_idata).a_artist
ax.vlines(artist_model['NrsBeforeDeath'], hdi.sel(hdi="lower"), hdi.sel(hdi="higher"), color="orange", alpha=0.5)

ax.set(
    xlabel="Nr songs in Top 2000 by artist before death",
    ylabel="Artist boost",
)
ax.legend(fontsize=10)


In [ ]:
outcomes = {}
days = np.arange(-365, 0)
for i, row in parameters.iterrows():

    recencyeffect = (np.exp(10**row['recency_effect_exponent'] * days) - np.exp(10**row['recency_effect_exponent'] * -365))* row['max_recency_effect']
    outcomes[i] = recencyeffect

In [ ]:
df_recencyeffect = pd.DataFrame(outcomes, index=np.arange(-365, 0)).apply(np.exp)

In [ ]:
%%time
ax = df_recencyeffect.sample(1500, axis='columns').plot(c='grey', alpha=0.01, legend=False)
ax = df_recencyeffect.median(axis='columns').plot(ax=ax, lw=3, c='darkblue')
plt.show()

In [ ]:
ax = df_recencyeffect.median(axis='columns').plot(lw=3, c='darkblue', label='Estimated effect')
(df_artist.assign(RecencyEffect = lambda df: df['Boost']/df['Boost'].median())
          .plot(x='DaysToStemperiode', y='RecencyEffect', kind='scatter', ax=ax, c='grey', label='Passed away artists')
)
ax.set_xlabel('Days until end of voting')
ax.set_ylabel('Extra boost')
plt.legend()
plt.tight_layout()
plt.gcf().savefig('RecencyEffect.jpg')

In [ ]:
df_recencyeffect.quantile([0.025, 0.16, 0.5, 0.84, 0.975], axis='columns').loc[:, [-365] + list(range(-350, -50, 50)) + list(range(-70, -10, 7)) + list(range(-10, 0, 2))]

In [ ]:
with multilevel_noncentered_model:
    ppc = pm.fast_sample_posterior_predictive(
                multilevel_noncentered_model_idata, random_seed=RANDOM_SEED, var_names=['y', 'za_artist']
                )


In [ ]:
predictions = pd.DataFrame(ppc['y'].T, index=df.index)

In [ ]:
za_artist = pd.DataFrame(ppc['za_artist'].T, index=artist_model.index) # The artist magic

In [ ]:
artist_outcomes = pd.concat([artist_model, za_artist.median(axis='columns').rename('ArtistEffect')], axis='columns')

In [ ]:
artist_outcomes.nlargest(25, 'ArtistEffect')[['NameArtist', 'ArtistEffect']]

In [ ]:
artist_outcomes.nsmallest(5, 'ArtistEffect')[['NameArtist', 'ArtistEffect']]

In [ ]:
trala = pd.concat([predictions.quantile([0.16, 0.5, 0.84], axis='columns').transpose(), df['LogBoost']], axis='columns').apply(np.exp)

In [ ]:
trala.describe()

In [ ]:
plt.errorbar(trala[0.5], trala['LogBoost'], xerr=[trala[0.5].sub(trala[0.16]), trala[0.84].sub(trala[0.5])], ls=' ', marker='o', alpha=0.2, ms=4)
plt.plot([0, 8], [0, 8], 'k--')
plt.ylabel('Boost')
plt.xlabel('Predicted boost')
plt.ylim(0, 6)
plt.xlim(0, 6)

In [ ]:
posterior = (df.reset_index().merge(artist_outcomes[['ArtistID', 'ArtistEffect']]).set_index('index')
              .assign(yhat = predictions.mean(axis='columns'),
                     error = lambda df: df['yhat'].sub(df['LogBoost']),
                     abserror = lambda df: df['error'].abs(),
                     error_without_artisteffect = lambda df: df['error'].sub(df['ArtistEffect']),
                     abserror_without_artisteffect = lambda df: df['error_without_artisteffect'].abs(),                   
                     )
            )

In [ ]:
posterior['LogBoost'].sub(posterior['LogBoost'].median()).abs().mean()

In [ ]:
posterior['abserror'].mean()

In [ ]:
posterior['abserror_without_artisteffect'].mean()

## Finding representative samples

In [ ]:
variables = [
             'DaysEffect',
            'LogPopularityNorm',
            'JarenGeleden',
            'MultiplePerformers',
            'LogSongPopularityWithinArtist',
            'PassingTooEarly',
            'IsDutchArtist',
             'LogBoost']
import sklearn.preprocessing
import sklearn.metrics
mm = sklearn.preprocessing.MinMaxScaler()

all_data = (posterior
            .assign(DaysEffect = lambda df: np.exp(0.007 * df['DaysToStemperiode']))
            .filter(variables)
            )
mm.fit(all_data)

data = (posterior
        .query('abserror < 0.15 & JarenGeleden > -19 & -0.15 < ArtistEffect < 0.15')
        .assign(DaysEffect = lambda df: np.exp(0.007 * df['DaysToStemperiode']))
        .filter(variables)
       )
data

In [ ]:
normalized_data = mm.transform(data)
dists = sklearn.metrics.pairwise_distances(normalized_data, metric='manhattan')
inds = np.argsort(dists, axis=None)[::-1]

In [ ]:
dists.max()

In [ ]:
boosting_effects = ['LogBoost', 'yhat', 'error', 'error_without_artisteffect']
variables = [v for v in variables if v != 'DaysEffect']
for ind in inds[:10:2]:
    
    x, y = divmod(ind, len(data))
    print(f'Distance: {dists[x, y]:.3f} {dists[y, x]:.3f}')
    ind = data.iloc[[x, y]].index

    new_df = pd.concat([df[['NameSong', 'Title', 'LogBoost', 'BoostSong', 'DaysToStemperiode'] + variables], posterior[['ArtistEffect']  + boosting_effects]], axis='columns')
    display(new_df.loc[ind])

## Figuring out the boost for two songs

We choose to compare Shaffy Cantate with Montserrat Caballé's Barcelona

In [ ]:
def calc_artist_boost(parameters, song):
    artist_boost = (parameters['a']
                    + parameters['effect_popularity'] * song['LogPopularityNorm']
                    + np.exp(parameters['recency_effect_exponent'] * song['DaysToStemperiode']) * parameters['max_recency_effect']
                    + parameters['history_effect'] * song['JarenGeleden']
                    )
    return artist_boost

def calc_song_boost(parameters, song):
    song_boost = (parameters['within_oeuvre_effect'] * song['LogSongPopularityWithinArtist']
          + parameters['sharing_effect'] * song['MultiplePerformers']
         )
    return song_boost

In [ ]:
def calc_boost(parameters, song):
    artist_boost = calc_artist_boost(parameters, song)
    song_boost = calc_song_boost(parameters, song)
    return artist_boost + song_boost

In [ ]:
parameters.agg(['median', 'mean'])

In [ ]:
def artist_effects(parameters, song):
    base_boost = np.exp(parameters['a'])
    history_effect = np.exp(parameters['history_effect'] * song['JarenGeleden'] )
    recency_effect = np.exp(np.exp(10**parameters['recency_effect_exponent'] * song['DaysToStemperiode']) * parameters['max_recency_effect'])
    popularity_effect = np.exp(parameters['effect_popularity'] * song['LogPopularityNorm'])
    dutch_effect = np.exp(parameters['is_dutch_effect'] * song['IsDutchArtist'])
    age_effect = np.exp(parameters['age_passing_effect'] * song['PassingTooEarly'])
    effects = {
                'Base': base_boost,
                'History': history_effect,
                'Popularity': popularity_effect,
                'Dutch': dutch_effect,
                'PassingTooEarly': age_effect,
                'Recency': recency_effect,
                }
    return effects

def song_effects(parameters, song):
    oeuvre_effect = np.exp(parameters['within_oeuvre_effect'] * song['LogSongPopularityWithinArtist'])
    sharing_effect = np.exp(parameters['sharing_effect'] * song['MultiplePerformers'])
    effects = {
                'WithinOeuvrePopularity': oeuvre_effect,
                'MultiplePerformers': sharing_effect
                }
    return effects

In [ ]:
effects_long = artist_effects(parameters.median(), df.loc[37989])
effects_cabballe = artist_effects(parameters.median(), df.loc[10901])
effects_kalverliefde = song_effects(parameters.median(), df.loc[37989])
effects_barcelona = song_effects(parameters.median(), df.loc[10901])

In [ ]:
def print_effects(effects, starting_point=1):
    effect = starting_point
    for name, size in effects.items():
        effect *= size
        print(f'The effect is {effect:0.2f} after {name} - distinct effect: {size:.2f}')
        

In [ ]:
print_effects(effects_long)
print('-')
print_effects(effects_kalverliefde, 1.56)
print('---')
print_effects(effects_cabballe)
print('-')
print_effects(effects_barcelona, 2.51)

In [ ]:
posterior.nlargest(10, 'error')#sort_values('LogBoost', ascending=False)

In [ ]:
%matplotlib inline
ax = pd.concat([df['LogBoost'], predictions.mean(axis='columns').rename('yhat')], axis='columns').plot(y='LogBoost', x='yhat', kind='scatter')
ax.plot([0, 3], [0, 3], 'k--')

In [ ]:
posterior[['LogBoost', 'yhat']].corr()